In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
import tensorflow as tf

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
random_state = 20

In [5]:
df = pd.read_csv('preprocessed_df.csv')
df = df.iloc[:,1:]

In [6]:
ydf = df['Survived']
xdf = df.drop(['Survived'],axis=1)
xtrain,xvalidation,ytrain,yvalidation = train_test_split(xdf,ydf,test_size = 0.2,random_state = random_state,shuffle = True)

In [7]:
Normalizer = tf.keras.layers.Normalization()
Normalizer.adapt(xdf)

In [8]:
summary = pd.DataFrame(xdf.keys().values.reshape(-1,1),columns=['variables'])
summary = pd.concat([summary,pd.DataFrame(Normalizer.mean.numpy().reshape(-1,1),columns=['mean'])],axis=1)

In [9]:
hidden_layer_size = 50
output_layer_size = 1
batch_size = 50
num_epochs= 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)


model = tf.keras.Sequential([
    Normalizer,
    tf.keras.layers.Dense(hidden_layer_size,activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation = 'relu'),
    tf.keras.layers.Dense(output_layer_size,activation = 'sigmoid')
])
model.compile(optimizer = 'adam',loss='LogCosh',metrics=['accuracy'])
model.fit(x = xtrain,
          y = ytrain,
          validation_data=[xvalidation,yvalidation],
          callbacks = [early_stopping],
          batch_size= batch_size,
          epochs=num_epochs,
          verbose=False)

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 18)               37        
 n)                                                              
                                                                 
 dense (Dense)               (None, 50)                950       
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 3,588
Trainable params: 3,551
Non-trainable params: 37
_________________________________________________________________


In [12]:
xtest = pd.read_csv('preprocessed_testdf.csv')

In [13]:
yhat = model.predict(xtest.iloc[:,1:])

14/14 [==============================] - 0s 1ms/step


In [14]:
weight = np.dot(np.dot(model.weights[3],model.weights[5]),model.weights[7])

In [15]:
summary = pd.concat([summary,pd.DataFrame(weight,columns=['coeffecients'])],axis=1)
summary = pd.concat([summary,pd.DataFrame(np.exp(weight),columns=['odds_ratio'])],axis=1)

In [16]:
summary.to_csv('variables_summary.csv')

In [26]:
prediction = pd.concat([xtest.iloc[:,0].copy(),pd.DataFrame(yhat,columns=['Survived'])],axis=1)
prediction.columns = ['passengerId','Survived']

In [34]:
prediction['Survived'] = prediction['Survived'].apply(lambda x: 1 if x>0.5 else 0)

In [37]:
prediction.to_csv('prediction.csv')